In [108]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [3]:
listing_url = "https://www.sgcarmart.com/used_cars/info.php?ID=862831&DL=3417"
response2 = requests.get(listing_url)
response2.status_code
page2 = response2.text
main_page2 = BeautifulSoup(page2, 'lxml')

### Listing URL Grabber

## Title/Brand Name

In [109]:
# Define a function to retrieve car brand name given a listing url

def brand_retrieval(parsed_url):
    brand_name = parsed_url.find(class_='link_redbanner').text.split()[0]
    return brand_name



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response = requests.get(listing_url)
response2 = requests.get(listing_url2)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')
print(brand_retrieval(parsed_listing_url))
print(brand_retrieval(parsed_listing_url2))

Honda
Subaru


### Price

In [110]:
def price_retrieval(parsed_listing_url):
    
    data_value = parsed_listing_url.find_all(class_='font_red')[0].text.strip()
    data_value = data_value.split('$')
    price = price_error_handling(data_value)
    return price


def price_error_handling(data_value):
    # Try-Exception error handling
    
    try:   # First try to deal with values higher than 1000
        price = data_value[1]  # will fail on IndexError if retrieves ['na'] scenario
        price = int(price.split(',')[0] + price.split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
        
    except IndexError:  # Dealing with ['na'] and ['', 900'] scenarios
        try: 
            price = int(data_value[1]) # Will fail on IndexError if ['na'] scenario
        except IndexError:  # Deals with ['na'] scenarios
            price = np.nan  # Stores NA values as nan
    
    return price


listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response = requests.get(listing_url)
response2 = requests.get(listing_url2)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')
print(price_retrieval(parsed_listing_url))
print(price_retrieval(parsed_listing_url2))


52888
10500


## Depreciation Value Per Year

In [111]:
# Define a function that retrieves depreciation value per year given a listing url
def depreciation_value_per_year_error_handler(data_value):
    if len(data_value) < 2:
        data_value = np.nan

    else: 
        data_value = data_value[1].split('/yr')
        try:                 
            desired_value = int(data_value[0].split(',')[0] +\
                                data_value[0].split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['900','']
        except IndexError: 
            desired_value = int(data_value[0])
        
        return desired_value
    
def depreciation_value_per_year_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_="label")[1].findNextSibling().text.strip().split('$')
    depreciation_value_per_year = depreciation_value_per_year_error_handler(data_value)
    return depreciation_value_per_year



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')

print(depreciation_value_per_year_retrieval(parsed_listing_url))
print(depreciation_value_per_year_retrieval(parsed_listing_url2))
    

5780
None


## Road Tax/Year

In [112]:
# Define a function to retrieve road tax /yr given a listing url
def road_tax_retrieval(parsed_listing_url):
    string_data = parsed_listing_url.find_all(class_='row_info')[1].text.strip()
    road_tax_yearly = road_tax_error_handler(string_data)
    
    return road_tax_yearly
    

def road_tax_error_handler(string_data):
    if '/yr' in string_data: # Only takes in scenarios that are not NA
        try:
            # Removes '$" character and splits string_data into a list of ['', 1,000] or ['', 900]
            road_tax_per_year = \
            string_data.replace('/yr','').strip().split('$') 

            # Accesses the second item in the list
            road_tax_per_year = road_tax_per_year[1] 


            road_tax_per_year = int(road_tax_per_year.split(',')[0] +\
                                    road_tax_per_year.split(',')[1])  # Will fail on IndexError if value is above 1000

        except IndexError: # Handles values that are below 1000. (i.e. ['',900])
            road_tax_pear_year = int(road_tax_per_year[1])

    else: # Deals with 'NA' scenario
        road_tax_per_year = np.nan
    
    return road_tax_per_year
        

    

listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')
    
    
print(road_tax_retrieval(parsed_listing_url))
print(road_tax_retrieval(parsed_listing_url2))

814
684


## Registered Date

In [113]:
# Define function to retrieve Registered date given a parsed listing url

# url to parse from "https://www.sgcarmart.com/used_cars/info.php?ID=862816&DL=2397"
# listing_url = "https://www.sgcarmart.com/used_cars/info.php?ID=862831&DL=3417"
# response2 = requests.get(listing_url)
# page2 = response2.text
# parsed_listing_url = BeautifulSoup(page2, 'lxml')


def registered_date_retrieval(parsed_listing_url):
    reg_date = parsed_listing_url.find_all(class_='row_bg')[1].find_all('td')[3].text.split()[0].split('(')[0]
    return reg_date



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(registered_date_retrieval(parsed_listing_url))
print(registered_date_retrieval(parsed_listing_url2))

15-Dec-2008
25-Feb-2010


## Days of COE Left

In [114]:
# Define a function to retrieve days of COE left in terms of days:

# Retrieve into usable format for link
def days_of_coe_retrieval(parsed_listing_url):
    """Takes in a listing page (individual cars) and retrieve the days of COE left
    ---
    Input: str
    Output: Days of COE left in integer form
    """
    days_of_coe_left_yy_mm_dd_format_for_cleaner_function=\
    parsed_listing_url.find_all(class_='row_bg')[1].find_all('td')[3].text.split('(')[1].split('COE')[0].strip()
    
    return yr_mm_dd_cleaner(days_of_coe_left_yy_mm_dd_format_for_cleaner_function)


# Define a function to calculate days of COE left
def yr_mm_dd_cleaner(str1):
    """Accepts a string that may or may include the elements yr mths days and 
    converts the whole string into number of days.
    ----
    Input: single string
    output: number of days in integer form
    ----
    Example string inputs:
    - 4yrs 2mths 23days
    - 5yrs
    - 2 mths 23 days
    - 50 days
    """
    
    # Convert days_of_coe_left_yy_mm_dd to days    
    year_index = str1.find('yr')
    if year_index == -1:
        year = 0
    else:
        year = int(str1[year_index-1])

        
    mth_index = str1.find('mth')
    if mth_index == -1:
        mth = 0
    else:
        mth = int(str1[mth_index-1])

        
    day_index = str1.find('day')
    if day_index == -1:
        day = 0
    else:
        day = int(str1[day_index-1])
       
    days_of_coe_left = (year * 365) + (mth * 30) + day 
    return days_of_coe_left



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(days_of_coe_retrieval(parsed_listing_url))
print(days_of_coe_retrieval(parsed_listing_url2))

3316
121


## Mileage in km

In [115]:
# Write a function to retrieve the mileage in km from a listjng url

def mileage_error_handler(data_value):
    if len(data_value) < 2:  # Deals with ['na'] scenarios
        mileage_km = np.nan  # Stores NA values as nan

    else:  
        try:                 
            mileage_km = int(data_value[0].strip().split(',')[0] + data_value[0].strip().split(',')[1])
        except IndexError: # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
            mileage_km = int(data_value[0].strip())
    
    return mileage_km

def mileage_retrieval(parsed_listing_url):
        
    data_value = parsed_listing_url.find_all(class_='row_info')[0].text.strip()
    data_value = data_value.split('km')
    mileage_km = mileage_error_handler(data_value)
    
    return mileage_km

listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(mileage_retrieval(parsed_listing_url)) 
print(mileage_retrieval(parsed_listing_url2))

55000
92000


## Manufactured Year

In [116]:
# Define a function that returns the manufactured date using a parsed html
def manufactured_year_retrieval(parsed_listing_url):
    manufactured_year = parsed_listing_url.find_all(class_='row_info')[6].text
    return manufactured_year



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(manufactured_year_retrieval(parsed_listing_url))
print(manufactured_year_retrieval(parsed_listing_url2))

2008
2009


## Transmission

In [117]:
# Define a function that returns the transmission based on a parsed listing url

def transmission_retrieval(parsed_listing_url):
    transmission = parsed_listing_url.find_all(class_='row_info')[7].text
    return transmission


listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(transmission_retrieval(parsed_listing_url))
print(transmission_retrieval(parsed_listing_url2))

Auto
Manual


## Dereg Value

In [118]:
# Write a function to retrieve dereg value from a parsed url

def dereg_value_retrieval(parsed_listing_url):
    # Splits into ['NA'], or ['$11,026', 'as', 'of', 'today', '(change)'] or ['$900', 'as', 'of', 'today', '(change)']
    data_value = parsed_listing_url.find_all(class_='row_info')[2].text.strip().split() 
    
    dereg_value_from_scrape_date = dereg_value_error_handler(data_value)
    return dereg_value_from_scrape_date
    

def dereg_value_error_handler(data_value):
    if len(data_value) < 2:  # Deals with ['NA'] scenario
        dereg_value_from_scrape_date = np.nan

    else: 
        data_value = data_value[0].split('$')[1] # Puts input into '11,026' or '900' format
        try:                 
            dereg_value_from_scrape_date = \
            int(data_value.split(',')[0] +\
                data_value.split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
        except IndexError: 
            dereg_value_from_scrape_date = int(data_value.strip())

        return dereg_value_from_scrape_date
 

listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')

print(dereg_value_retrieval(parsed_listing_url))
print(dereg_value_retrieval(parsed_listing_url2))

25229
7978


## OMV

In [119]:
# Define a function that retrieves omv based on a parsed listing url
def omv_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        omv = np.nan

    else:
        try:
            omv = int(data_value[1].split(',')[0] +\
                      data_value[1].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            omv = int(data_value[1])
    return omv


def omv_retrieval(parsed_listing_url):    
    data_value = parsed_listing_url.find_all(class_='row_info')[8].text.split('$') 
    # Splits data into ['', '21,967'], ['','900'] or ['NA'] format for input into error function
    
    omv = omv_error_handler(data_value)
    return omv     




listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(omv_retrieval(parsed_listing_url))
print(omv_retrieval(parsed_listing_url2))

20407
14597


## ARF

In [120]:
# Write a function that retrieves ARF based on a parsed listing url
def error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[1].split(',')[0] +\
                                data_value[1].split(',')[1])   # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[1])
    return desired_value


def arf_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[9].text.split('$')
    arf = error_handler(data_value)
    return arf



listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')



print(arf_retrieval(parsed_listing_url))
print(arf_retrieval(parsed_listing_url2))
    

20407
14597


## COE Price as of Today

In [121]:
# Write a function to retrieve COE as of today from a parsed listing url
def coe_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        coe_from_scrape_date = np.nan

    else:
        try:
            coe_from_scrape_date = int(data_value[1].split(',')[0] +\
                                       data_value[1].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            coe_from_scrape_date = int(data_value[1])
    return coe_from_scrape_date


def coe_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[3].text.split('$')
    coe_from_scrape_date = coe_error_handler(data_value)
    return coe_from_scrape_date
  
    

listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')

    
print(coe_retrieval(parsed_listing_url))
print(coe_retrieval(parsed_listing_url2))

27571
19989


# Engine Capacity (CC)

In [122]:
def engine_capacity_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[0].split(',')[0] +\
                                       data_value[0].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[0])
    return desired_value


def engine_capacity_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[4].text.strip().split('cc')
    engine_capacity = engine_capacity_error_handler(data_value)
    return engine_capacity


listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(engine_capacity_retrieval(parsed_listing_url))
print(engine_capacity_retrieval(parsed_listing_url2))

1595
1498


## Power (kW) Something to consider for the future. No time already
refer to car_webscraper if you want to work on it

## Curb Weight

In [123]:
def curb_weight_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[0].split(',')[0] +\
                                       data_value[0].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[0])
    return desired_value


def curb_weight_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[5].text.split()
    curb_weight = curb_weight_error_handler(data_value)
    return curb_weight


listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')


print(curb_weight_retrieval(parsed_listing_url))
print(curb_weight_retrieval(parsed_listing_url2))

1220
1310


## No. Of Owners

In [124]:
# Define a function to retrieve the no of owners from a parsed listing url
def number_of_owners_retrieval(parsed_listing_url):
    no_of_owners = int(parsed_listing_url.find_all(class_='row_info')[-1].text)
    return no_of_owners


listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')

print(number_of_owners_retrieval(parsed_listing_url))
print(number_of_owners_retrieval(parsed_listing_url2))

3
2


## Type of Vehicle

In [125]:
# Define a function that returns the type of vehicle given a parsed listing url

def type_of_vehicle_retrieval(parsed_listing_url):
    type_of_vehicle = parsed_listing_url.find(class_='row_bg1').find_all('a')[0].text 
    return type_of_vehicle

listing_url = 'https://www.sgcarmart.com/used_cars/info.php?ID=863352&DL=1000'
response = requests.get(listing_url)
parsed_listing_url = BeautifulSoup(response.text, 'lxml')

listing_url2 = 'https://www.sgcarmart.com/used_cars/info.php?ID=863354&DL=1281'
response2 = requests.get(listing_url2)
parsed_listing_url2 = BeautifulSoup(response2.text, 'lxml')

print(type_of_vehicle_retrieval(parsed_listing_url))
print(type_of_vehicle_retrieval(parsed_listing_url2))

Mid-Sized Sedan
Mid-Sized Sedan


# WebScraping

In [143]:
# Creating a list of main pages to iterate through
main_page_listing_list = []
for idx, link in enumerate(range(40)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2"
    main_page_listing_list.append(url)

In [144]:
print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

## Retrieving individual listing urls from main pages of 100 listings

In [145]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # Find every single URL in the webpage , refer to this post: # https://stackoverflow.com/questions/46490626/getting-all-links-from-a-page-beautiful-soup
    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(2)

In [146]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

3997
3997
3997


In [147]:
print(listing_urls[:5])

['https://www.sgcarmart.com/used_cars/info.php?ID=862892&DL=511', 'https://www.sgcarmart.com/used_cars/info.php?ID=857825&DL=1000', 'https://www.sgcarmart.com/used_cars/info.php?ID=858133&DL=2870', 'https://www.sgcarmart.com/used_cars/info.php?ID=859437&DL=1292', 'https://www.sgcarmart.com/used_cars/info.php?ID=862543&DL=1000']


## DataFrame Creation

In [148]:
# Creating an empty DataFrame for attributes of interest
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

In [139]:
file_name = 'sgcarmart_used_cars_prices'
i = 0 # Indexing rows in the DF

for listingurl in listing_url:
    response = requests.get(listingurl)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
    df.to_csv("{}.csv".format(filename))    
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(2)  # Prevents us from getting locked out of the website
    

In [140]:
df = pd.read_csv('test_file.csv',index_col=0)
df

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE
0,https://www.sgcarmart.com/used_cars/info.php?I...,Chevrolet,NaN,NaN,18-Sep-2014,10307.0,2014,604,Auto,42336.0,23/10/2019,15016.0,10016.0,71990.0,1465.0,1362.0,1440.0,1.0,Mid-Sized Sedan
1,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,90888.0,23950.0,07-Mar-2012,89000.0,2011,2260,Auto,56856.0,23/10/2019,68256.0,68256.0,67101.0,851.0,2893.0,1440.0,3.0,Sports Car
2,https://www.sgcarmart.com/used_cars/info.php?I...,Volkswagen,105988.0,10580.0,30-Jan-2018,32800.0,2017,976,Auto,63020.0,23/10/2019,32116.0,36963.0,42661.0,3016.0,1798.0,1480.0,1.0,Luxury Sedan
3,https://www.sgcarmart.com/used_cars/info.php?I...,Honda,48200.0,8430.0,19-Mar-2015,76800.0,2015,684,Auto,37151.0,23/10/2019,15274.0,5274.0,61410.0,1948.0,1498.0,1046.0,3.0,Hatchback
4,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,108800.0,10880.0,21-Oct-2009,120000.0,2009,3322,Auto,37502.0,23/10/2019,69278.0,69278.0,37502.0,3322.0,3598.0,2170.0,4.0,SUV
5,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,41800.0,8490.0,24-Sep-2009,88000.0,2009,1210,Auto,18680.0,23/10/2019,27271.0,27271.0,18971.0,1490.0,1998.0,1530.0,2.0,Luxury Sedan
6,https://www.sgcarmart.com/used_cars/info.php?I...,Audi,84800.0,10680.0,23-Dec-2016,51287.0,2016,392,Auto,47984.0,23/10/2019,24583.0,16417.0,49751.0,2594.0,999.0,1280.0,1.0,Luxury Sedan
7,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,195000.0,17350.0,21-Mar-2019,NaN,2018,1792,Auto,86039.0,23/10/2019,50698.0,63257.0,41000.0,3410.0,2493.0,2030.0,1.0,MPV
8,https://www.sgcarmart.com/used_cars/info.php?I...,Mazda,58800.0,7200.0,21-Mar-2017,48000.0,2017,682,Auto,45084.0,23/10/2019,15951.0,10951.0,49751.0,2680.0,1496.0,1310.0,2.0,Mid-Sized Sedan
9,https://www.sgcarmart.com/used_cars/info.php?I...,Mazda,46800.0,9350.0,23-Nov-2009,119009.0,2009,684,Auto,NaN,23/10/2019,17571.0,17571.0,NaN,1825.0,1498.0,1041.0,2.0,Hatchback


In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
datetime.now().strftime("%d/%m/%Y, %H:%M:%S")

In [50]:
def attribute_scraper4_shorter_time(input_listing_url):
    """Accepts a list of individual car listing urls and returns variables of importance for linear regression analysis.
    ---
    Input: List of urls
    Output: Variables
    """
    
    # Parsing an individual car listing url
    response = requests.get(input_listing_url)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    
    i = 0 # Indexing rows in the DF
    
       
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listing_url
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan
    
    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan
        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan
        
    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
        
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(2)               

In [26]:
def attribute_scraper(listing_url_list):
    """Accepts a list of individual car listing urls and returns variables of importance for linear regression analysis.
    ---
    Input: List of urls
    Output: Variables
    """
    
    # Retrieval functions
    print(f"Brand: {brand_retrieval(listing_url)}\n")
    print(f"Price: {price_retrieval(listing_url)}\n")
    print(f"Depreciation Value Per Year: {depreciation_value_per_year_retrieval(listing_url)}\n")
    print(f"Registered Date: {registered_date_retrieval(listing_url)}\n")
    print(f"Mileage (km): {mileage_retrieval(listing_url)}\n")
    print(f"Manufactured Year: {manufactured_year_retrieval(listing_url)}\n")
    print(f"Road Tax Per Year:{road_tax_retrieval(listing_url)}\n")
    print(f"Transmission: {transmission_retrieval(listing_url)}\n")
    print("Dereg Value as of {}: {}".format(datetime.now().strftime("%d/%m/%Y, %H:%M:%S"),
                                            dereg_value_retrieval(listing_url)))
    print(f"OMV: {omv_retrieval(listing_url)}\n")
    print(f"ARF: {arf_retrieval(listing_url)}\n")
    print(f"COE as of Today: {coe_retrieval(listing_url)}\n")
    print(f"Days of COE Left: {days_of_coe_retrieval(listing_url)}\n")
    print(f"Engine Capacity (CC):{ engine_capacity_retrieval(listing_url)}\n")
    print(f"Curb Weight (kg): {curb_weight_retrieval(listing_url)}\n")
    print(f"# Of Owners: {number_of_owners_retrieval(listing_url)}\n")
    print(f"Vehicle Type:{type_of_vehicle_retrieval(listing_url)}\n")
                                        
                                

In [27]:
def attribute_scraper2(listing_url_list):
    """Accepts a list of individual car listing urls and returns variables of importance for linear regression analysis.
    ---
    Input: List of urls
    Output: Variables
    """
    i = 0
    # Retrieval functions
    df.loc[i, 'LISTING_URL'] = listing_url
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except ValueError:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
    df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
                                        
                                

In [29]:
def attribute_scraper3(listing_url_list):
    """Accepts a list of individual car listing urls and returns variables of importance for linear regression analysis.
    ---
    Input: List of urls
    Output: Variables
    """
    i = 0
    # Retrieval functions
    df.loc[i, 'LISTING_URL'] = listing_url
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan
    
    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan
        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan
        
    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
                                        
                                